<a href="https://colab.research.google.com/github/Chu-Yichen/INST0001_Database-System/blob/main/Project_%26_Budget.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import random
from datetime import datetime, timedelta

# Return a random datetime between two dates.
def random_date(start, end):
    delta = end - start
    return start + timedelta(days=random.randint(0, delta.days))

# Generate project start and end dates.
def generate_project_dates(year):
    """
    Generate a random start_date within the given year and an end_date.
    The project lasts at least 6 months and ends no later than December 31, 2024.
    For 2024, the start date is chosen no later than December 31 minus 180 days.
    Returns start_date and end_date as 'YYYY-MM-DD' strings.
    """
    start_range_start = datetime(year, 1, 1)
    start_range_end = datetime(year, 12, 31) - timedelta(days=180) if year == 2024 else datetime(year, 12, 31)
    start_dt = random_date(start_range_start, start_range_end)

    end_range_start = start_dt + timedelta(days=180)
    end_range_end = datetime(2024, 12, 31)
    end_dt = random_date(end_range_start, end_range_end)

    return start_dt.strftime("%Y-%m-%d"), end_dt.strftime("%Y-%m-%d")

# Determine the number of projects based on region type.
def get_project_count(region_type):
    """
    Urban regions: 1-2 projects per year.
    Rural regions: 3-4 projects per year.
    """
    return random.randint(1, 2) if region_type == "urban" else random.randint(3, 4)

# Randomly choose an SDG target and its associated category.
def choose_target_and_category():
    """
    Randomly selects one target indicator and then chooses the project category using specified probabilities [We restrict categories here to enhance realism]:
      - For target "1.1.1": 50% 'Financial Aid', 50% 'Education'
      - For target "4.6.1": 20% 'Advertising Campaign', 80% 'Education'
      - For target "5.4.1": 60% 'Advertising Campaign', 40% 'Education'
    Returns (target, category) with target as e.g. "1.1.1".
    """
    target = random.choice(["1.1.1", "4.6.1", "5.4.1"])
    if target == "1.1.1":
        category = random.choices(["Financial Aid", "Education"], weights=[0.5, 0.5])[0]
    elif target == "4.6.1":
        category = random.choices(["Advertising Campaign", "Education"], weights=[0.2, 0.8])[0]
    else:  # target == "5.4.1"
        category = random.choices(["Advertising Campaign", "Education"], weights=[0.6, 0.4])[0]
    return target, category

# Generate PROJECT records for each region from 2019-2024.
def generate_projects(regions, years):
    """
    For each region and each year (2019-2024), generate project records.
    Each project includes the following attributes:
      - project_id (primary key)
      - project_name (e.g., "Project_1")
      - category (the chosen project category)
      - start_date and end_date (with at least a 6-month duration, ending by 2024-12-31)
      - targets_sdg_indicator (e.g., "1.1.1")
      - takes_place_in_region_id (foreign key)
    """
    projects = []
    project_id = 1
    for region in regions:
        for year in years:
            count = get_project_count(region["region_type"])
            for _ in range(count):
                start_date, end_date = generate_project_dates(year)
                target, category = choose_target_and_category()
                project = {
                    "project_id": project_id,
                    "project_name": f"Project_{project_id}",
                    "category": category,
                    "start_date": start_date,
                    "end_date": end_date,
                    "targets_sdg_indicator": target,
                    "takes_place_in_region_id": region["region_id"]
                }
                projects.append(project)
                project_id += 1
    return projects


# Allocate budgets for each project and create BUDGET records.
def allocate_budgets(projects):
    """
    For each project, randomly choose a total_budget from the allowed set:[500K, 600K, 700K, 800K, 900K, 1M]
    Generate an expenditure as a random integer between 50% and 100% of the total_budget.
    Returns a list of budget records for the BUDGET table.
    """
    allowed_budgets = [500_000, 600_000, 700_000, 800_000, 900_000, 1_000_000]
    budget_records = []
    for p in projects:
        total_budget = random.choice(allowed_budgets)
        expenditure = random.randint(int(total_budget * 0.5), total_budget)
        budget_records.append({
            "project_id": p["project_id"],
            "expenditure": expenditure,
            "total_budget": total_budget
        })
    return budget_records


# Write PROJECT table data to CSV.
def write_projects_to_csv(projects, filename='projects.csv'):
    """
    Write PROJECT table data to a CSV file.
    Columns: project_id, project_name, category, start_date, end_date, targets_sdg_indicator, takes_place_in_region_id.
    """
    fieldnames = ["project_id", "project_name", "category", "start_date", "end_date", "targets_sdg_indicator", "takes_place_in_region_id"]
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for p in projects:
            row = {k: p[k] for k in fieldnames}
            writer.writerow(row)


# Write BUDGET table data to CSV.
def write_budget_to_csv(budget_records, filename='budget.csv'):
    """
    Write BUDGET table data to a CSV file.
    Expected columns: project_id, expenditure, total_budget.
    """
    fieldnames = ["project_id", "expenditure", "total_budget"]
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(budget_records)


# Main function to generate and write data.
def main():
    regions = [
        {"region_id": "SSD001", "region_type": "urban", "region_name": "Juba", "country_id": "SOU"},
        {"region_id": "SSD002", "region_type": "urban", "region_name": "Wau", "country_id": "SOU"},
        {"region_id": "SSD003", "region_type": "rural", "region_name": "Bor", "country_id": "SOU"},
        {"region_id": "SSD004", "region_type": "rural", "region_name": "Yambio", "country_id": "SOU"},
        {"region_id": "RCB001", "region_type": "urban", "region_name": "Kinshasa", "country_id": "DEM"},
        {"region_id": "RCB002", "region_type": "urban", "region_name": "Lubumbashi", "country_id": "DEM"},
        {"region_id": "RCB003", "region_type": "rural", "region_name": "Goma", "country_id": "DEM"},
        {"region_id": "RCB004", "region_type": "rural", "region_name": "Bukavu", "country_id": "DEM"},
        {"region_id": "PSE001", "region_type": "urban", "region_name": "Ramallah", "country_id": "PAL"},
        {"region_id": "PSE002", "region_type": "urban", "region_name": "Gaza", "country_id": "PAL"},
        {"region_id": "PSE003", "region_type": "rural", "region_name": "Jenin", "country_id": "PAL"},
        {"region_id": "PSE004", "region_type": "rural", "region_name": "Tubas", "country_id": "PAL"},
        {"region_id": "YEM001", "region_type": "urban", "region_name": "Sanaa", "country_id": "YEM"},
        {"region_id": "YEM002", "region_type": "urban", "region_name": "Aden", "country_id": "YEM"},
        {"region_id": "YEM003", "region_type": "rural", "region_name": "Ibb", "country_id": "YEM"},
        {"region_id": "YEM004", "region_type": "rural", "region_name": "Taiz", "country_id": "YEM"},
        {"region_id": "LEB001", "region_type": "urban", "region_name": "Beirut", "country_id": "LEB"},
        {"region_id": "LEB002", "region_type": "urban", "region_name": "Tripoli", "country_id": "LEB"},
        {"region_id": "LEB003", "region_type": "rural", "region_name": "Baalbek", "country_id": "LEB"},
        {"region_id": "LEB004", "region_type": "rural", "region_name": "Byblos", "country_id": "LEB"}
    ]

    years = list(range(2019, 2025))
    projects = generate_projects(regions, years)
    budget_records = allocate_budgets(projects)

    write_projects_to_csv(projects, filename='projects.csv')
    write_budget_to_csv(budget_records, filename='budget.csv')

if __name__ == '__main__':
    main()
